In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
import numpy as np
import os
import json

# Define the path to your dataset
path_to_dataset = 'dataset/'

all_files = tf.data.Dataset.list_files(os.path.join(path_to_dataset, '*.jpg'))
all_files = all_files.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(tf.reshape(x, [-1])))
dataset = all_files


In [5]:

#Use the map method to load and decode the images
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

#Load VGG16 model
model = VGG16(weights='imagenet')

#Function to extract features from an image using VGG16
def extract_features(image):
    image_batch = tf.expand_dims(image, axis=0)
    features = model.predict(image_batch)
    return features[0]
    

In [6]:
x_train_features = [extract_features(img) for img in dataset]


1/1 [==============================] - 0s 268ms/step


In [10]:
all_files_np = np.array(list(all_files.as_numpy_iterator()))
filenames = all_files_np.tolist()
m = list(map(lambda x: x.decode(), filenames))
features = [np_array.tolist() for np_array in x_train_features]
dict_final = {"filenames" : m,
                    "features" : features}
                    
with open('features_VGG16.json', 'w') as f:
    json.dump(dict_final, f)

TypeError: Object of type bytes is not JSON serializable